In [1]:
import numpy as np
from scipy.stats import norm
import pandas as pd
from scipy.interpolate import CubicSpline
import matplotlib.pyplot as plt

In [2]:

def d1(S, K, T, rd, rf, sigma):
    return (np.log(S / K) + (rd - rf + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))

def d2(S, K, T, rd, rf, sigma):
    return d1(S, K, T, rd, rf, sigma) - sigma * np.sqrt(T)

In [3]:

def A1(S, K, T, rd, rf, sigma, a, x1):
    return np.exp(-rf * T) * a * S * norm.cdf(a * x1) - np.exp(-rd * T) * K * norm.cdf(a * (x1 - sigma * np.sqrt(T)))

def A2(S, K, T, rd, rf, sigma, a, x1, x2):
    return np.exp(-rf * T) * a * S * norm.cdf(a * x1) - np.exp(-rd * T) * K * norm.cdf(a * (x2 - sigma * np.sqrt(T)))

def A3(S, K, T, rd, rf, sigma, a, b, B, x3, mu):
    return np.exp(-rf * T) * a * (B / S) ** (2 * (mu + 1)) * norm.cdf(b * x3) - np.exp(-rd * T) * K * (B / S) ** (2 * mu) * norm.cdf(b * (x3 - sigma * np.sqrt(T)))

def A4(S, K, T, rd, rf, sigma, a, b, B, x4, mu):
    return np.exp(-rf * T) * a *( (B / S) ** (2 * (mu + 1))) * norm.cdf(b * x4) - np.exp(-rd * T) * K * (B / S) ** (2 * mu) * norm.cdf(b * (x4 - sigma * np.sqrt(T)))

def A5(K, T, rd, b, B, S, x2, x4, mu, sigma):
    return K * np.exp(-rd * T) * (norm.cdf(b * (x2 - sigma * np.sqrt(T))) - (B / S) ** (2 * mu) * norm.cdf(b * (x4 - sigma * np.sqrt(T))))

def A6(K, b, B, S, x5, mu, lambda_param, sigma, T):
    return K * ((B / S) ** (mu + lambda_param) * norm.cdf(b * x5) + ((B / S) ** (mu - lambda_param)) * norm.cdf(b * (x5 - 2 * lambda_param * sigma * np.sqrt(T))))


In [4]:
def calculate_x(S, K, B, T, sigma, rd, rf):
    mu = (rd - rf - 0.5 * sigma ** 2) / sigma ** 2
    lambda_param = np.sqrt((mu ** 2) + 2 * rd / (sigma ** 2))

    x1 = (np.log(S / K) + (rd - rf + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    x2 = (np.log(S / B) + (rd - rf + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    x3 = (np.log(B ** 2 / (S * K)) + (rd - rf + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    x4 = (np.log(B / S) + (rd - rf + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    x5 = (np.log(B / S) + lambda_param * (sigma ** 2) * T) / (sigma * np.sqrt(T))

    return x1, x2, x3, x4, x5, mu, lambda_param

In [5]:
def single_barrier_option_price(S, K, B, T, rd, rf, sigma, option_type, barrier_type):
    a = 1 if option_type == 'call' else -1
    b = 1 if barrier_type == 'out' else -1
    
    x1, x2, x3, x4, x5, mu, lambda_param = calculate_x(S, K, B, T, sigma, rd, rf)
      
    if option_type == 'call' and barrier_type == 'down and in':
        if K <= B:
            return A1(S, K, T, rd, rf, sigma, a, x1) - A2(S, K, T, rd, rf, sigma, a, x1, x2) + A4(S, K, T, rd, rf, sigma, a, b, B, x4, mu) + A5(K, T, rd, b, B, S, x2, x4, mu, sigma)
        else:
            return A3(S, K, T, rd, rf, sigma, a, b, B, x3, mu) + A5(K, T, rd, b, B, S, x2, x4, mu, sigma)
    elif option_type == 'call' and barrier_type == 'up and in':
        if K <= B:
            return A2(S, K, T, rd, rf, sigma, a, x1, x2) - A3(S, K, T, rd, rf, sigma, a, b, B, x3, mu) + A4(S, K, T, rd, rf, sigma, a, b, B, x4, mu) + A5(K, T, rd, b, B, S, x2, x4, mu, sigma)
        else:
            return A1(S, K, T, rd, rf, sigma, a, x1) + A5(K, T, rd, b, B, S, x2, x4, mu, sigma)
    elif option_type == 'put' and barrier_type == 'down and in':
        if K <= B:
            return A1(S, K, T, rd, rf, sigma, a, x1) + A5(K, T, rd, b, B, S, x2, x4, mu, sigma)
        else:
            return A2(S, K, T, rd, rf, sigma, a, x1, x2) - A3(S, K, T, rd, rf, sigma, a, b, B, x3, mu) + A4(S, K, T, rd, rf, sigma, a, b, B, x4, mu) + A5(K, T, rd, b, B, S, x2, x4, mu, sigma)
    elif option_type == 'put' and barrier_type == 'up and in':
        if K <= B:
            return A3(S, K, T, rd, rf, sigma, a, b, B, x3, mu) + A5(K, T, rd, b, B, S, x2, x4, mu, sigma)
        else:
            return A1(S, K, T, rd, rf, sigma, a, x1) - A2(S, K, T, rd, rf, sigma, a, x1, x2) + A4(S, K, T, rd, rf, sigma, a, b, B, x4, mu) + A5(K, T, rd, b, B, S, x2, x4, mu, sigma)
    elif option_type == 'call' and barrier_type == 'down and out':
        if K <= B:
            return A2(S, K, T, rd, rf, sigma, a, x1, x2) - A4(S, K, T, rd, rf, sigma, a, b, B, x4, mu) + A6(K, b, B, S, x5, mu, lambda_param, sigma, T)
        else:
            return A1(S, K, T, rd, rf, sigma, a, x1) - A3(S, K, T, rd, rf, sigma, a, b, B, x3, mu) + A6(K, b, B, S, x5, mu, lambda_param, sigma, T)
    elif option_type == 'call' and barrier_type == 'up and out':
        if K <= B:
            return A1(S, K, T, rd, rf, sigma, a, x1) - A2(S, K, T, rd, rf, sigma, a, x1, x2) + A3(S, K, T, rd, rf, sigma, a, b, B, x3, mu) - A4(S, K, T, rd, rf, sigma, a, b, B, x4, mu) + A6(K, b, B, S, x5, mu, lambda_param, sigma, T)
        else:
            return A6(K, b, B, S, x5, mu, lambda_param, sigma, T)
    elif option_type == 'put' and barrier_type == 'down and out':
        if K <= B:
            return A6(K, b, B, S, x5, mu, lambda_param, sigma, T)
        else:
            return A1(S, K, T, rd, rf, sigma, a, x1) - A2(S, K, T, rd, rf, sigma, a, x1, x2) + A3(S, K, T, rd, rf, sigma, a, b, B, x3, mu) - A4(S, K, T, rd, rf, sigma, a, b, B, x4, mu) + A6(K, b, B, S, x5, mu, lambda_param, sigma, T)
    elif option_type == 'put' and barrier_type == 'up and out':
        if K <= B:
            return A1(S, K, T, rd, rf, sigma, a, x1) - A3(S, K, T, rd, rf, sigma, a, b, B, x3, mu) + A6(K, b, B, S, x5, mu, lambda_param, sigma, T)
        else:
            return A2(S, K, T, rd, rf, sigma, a, x1, x2) - A4(S, K, T, rd, rf, sigma, a, b, B, x4, mu) + A6(K, b, B, S, x5, mu, lambda_param, sigma, T)
    else:
        return None


In [6]:
T = 1
data = {
    'exp_in_yr': [0.00274, 0.019178, 0.038356, 0.079245, 0.083333, 0.166667, 0.25, 0.333333, 0.416667, 0.5, 0.75, 1, 1.5, 2, 3, 4, 5, 6, 7, 10, 15, 20, 25, 30],
    '10D_Call': [4.4175, 3.895, 3.9325, 3.9925, 4.10875, 4.285, 4.3975, 4.6325, 4.75825, 4.86, 5.075, 5.2675, 5.55125, 5.71875, 5.99125, 6.22875, 6.485, 6.4925, 6.4975, 6.46375, 6.8925, 6.9175, 6.9385, 6.9575],
    '25D_Call': [3.4275, 2.905, 2.92625, 3.00625, 3.11625, 3.2575, 3.385, 3.61625, 3.7365, 3.8325, 4.0575, 4.275, 4.5025, 4.64, 4.88125, 5.09, 5.275, 5.35675, 5.41375, 5.3825, 5.495, 5.164, 5.1635, 5.16275],
    'ATM': [2.565, 2.215, 2.2275, 2.325, 2.43, 2.5775, 2.7075, 2.9475, 3.0785, 3.1775, 3.4025, 3.5925, 3.8175, 3.9575, 4.1625, 4.3675, 4.5325, 4.611, 4.665, 4.635, 4.5275, 4.527, 4.526, 4.5255],
    '75D_Call': [2.6525, 2.125, 2.12875, 2.23375, 2.34375, 2.4925, 2.635, 2.87875, 3.0095, 3.1075, 3.3625, 3.54, 3.7875, 3.935, 4.17375, 4.38, 4.565, 4.65725, 4.72125, 4.6875, 4.795, 4.762, 4.7605, 4.75925],
    '90D_Call': [2.7525, 2.295, 2.2975, 2.4075, 2.51625, 2.68, 2.8325, 3.0875, 3.22575, 3.33, 3.57, 3.7975, 4.11875, 4.29125, 4.55875, 4.78625, 5.035, 5.1165, 5.1725, 5.13125, 5.6425, 5.6275, 5.6135, 5.6015]
}

df = pd.DataFrame(data)

# Perform cubic spline interpolation for each delta
deltas = ['10D_Call', '25D_Call', 'ATM', '75D_Call', '90D_Call']
cs_results = {}

for delta in deltas:
    cs = CubicSpline(df['exp_in_yr'], df[delta])
    cs_results[delta] = cs(T)

# Display the interpolated volatilities for the given expiry
cs_results

{'10D_Call': array(5.2675),
 '25D_Call': array(4.275),
 'ATM': array(3.5925),
 '75D_Call': array(3.54),
 '90D_Call': array(3.7975)}

In [7]:
# Create a DataFrame from the cubic spline interpolation results
cs_df = pd.DataFrame({
    'Delta': [0.1, 0.25, 0.5, 0.75, 0.9],
    'Volatility': [cs_results[delta] for delta in deltas]
})

# Perform cubic spline interpolation on this DataFrame
cs_volatility = CubicSpline(cs_df['Delta'], cs_df['Volatility'])

# Function to get volatility for a given delta
def get_volatility_for_delta(delta):
    return cs_volatility(delta)

# Example usage for a given delta (e.g., 0.3)
example_delta = 0.5
volatility_for_example_delta = get_volatility_for_delta(example_delta)
volatility_for_example_delta


array(3.5925)

In [8]:
# Example usage:
S = 100  # Spot price
K = 90  # Strike price
B = 95  # Barrier level
T = 1  # Time to maturity
rd = 0.0622 # Domestic risk-free rate
rf = 0.05  # Foreign risk-free rate
sigma = 0.3  # Volatility
option_type = 'call'
barrier_type = 'down and in'

price = single_barrier_option_price(S, K, B, T, rd, rf, sigma, option_type, barrier_type)
print(f"The price of the single barrier option is: {price}")

# print each of A values used in the calculation of the option price of call option with down and in barrier
a = 1 if option_type == 'call' else -1
b = 1 if barrier_type == 'out' else -1
x1, x2, x3, x4, x5, mu, lambda_param = calculate_x(S, K, B, T, sigma, rd, rf)
print(f"A1: {A1(S, K, T, rd, rf, sigma, a, x1)}")
print(f"A2: {A2(S, K, T, rd, rf, sigma, a, x1, x2)}")
print(f"A3: {A3(S, K, T, rd, rf, sigma, a, b, B, x3, mu)}")
print(f"A4: {A4(S, K, T, rd, rf, sigma, a, b, B, x4, mu)}")
print(f"A5: {A5(K, T, rd, b, B, S, x2, x4, mu, sigma)}")
print(f"A6: {A6(K, b, B, S, x5, mu, lambda_param, sigma, T)}")

# print the price of the single call down and in barrier option
print(f"The price of the single call down and in barrier option is: {A1(S, K, T, rd, rf, sigma, a, x1) - A2(S, K, T, rd, rf, sigma, a, x1, x2) + A4(S, K, T, rd, rf, sigma, a, b, B, x4, mu) + A5(K, T, rd, b, B, S, x2, x4, mu, sigma)}")

# print the price of the single call down and out barrier option
print(f"The price of the single call down and out barrier option is: {A2(S, K, T, rd, rf, sigma, a, x1, x2) - A4(S, K, T, rd, rf, sigma, a, b, B, x4, mu) + A6(K, b, B, S, x5, mu, lambda_param, sigma, T)}")



The price of the single barrier option is: -72.53268093128213
A1: 16.79318378437565
A2: 22.796309472576517
A3: -47.021856476730285
A4: -53.149348371865244
A5: -13.38020687121602
A6: 105.18605320576911
The price of the single call down and in barrier option is: -72.53268093128213
The price of the single call down and out barrier option is: 181.13171105021087


In [9]:
# find price of a normal call option with above given parameters not barrier option, just vanilla call option

def vanilla_option_price(S, K, T, rd, rf, sigma, option_type='call'):
    d1 = (np.log(S / K) + (rd - rf + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        price = S * np.exp(-rf * T) * norm.cdf(d1) - K * np.exp(-rd * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-rd * T) * norm.cdf(-d2) - S * np.exp(-rf * T) * norm.cdf(-d1)
    
    return price
    
vanilla_price = vanilla_option_price(S, K, T, rd, rf, sigma, option_type)
print(f"The price of the vanilla option is: {vanilla_price}")


The price of the vanilla option is: 16.79318378437565


In [10]:
def down_and_out_call(S, K, T, rd, rf, sigma, B):
    # Calculate the vanilla call option price
    vanilla_price = vanilla_option_price(S, K, T, rd, rf, sigma, 'call')
    
    # Calculate the parameter mu
    mu = (rd - rf - 0.5 * sigma**2) / sigma**2
    
    # Calculate the adjustment term for the down-and-out call
    term2 = (B / S)**(2 * (mu + 1)) * vanilla_option_price(B**2 / S, K, T, rd, rf, sigma, 'call')
    
    # Return the down-and-out call price
    return max(vanilla_price - term2,0)

def down_and_in_call(S, K, T, rd, rf, sigma, B):
    vanilla_price = vanilla_option_price(S, K, T, rd, rf, sigma, 'call')
    down_and_out_price = down_and_out_call(S, K, T, rd, rf, sigma, B)
    return vanilla_price - down_and_out_price


In [11]:
# for up and out call option  and up and in call option

def up_and_out_call(S, K, T, rd, rf, sigma, B):
    vanilla_price = vanilla_option_price(S, K, T, rd, rf, sigma, 'call')
    mu = (rd - rf - 0.5 * sigma**2) / sigma**2
    term2 = (B / S)**(2 * mu) * vanilla_option_price(B, K, T, rd, rf, sigma, 'call')
    return max(vanilla_price - term2,0)

def up_and_in_call(S, K, T, rd, rf, sigma, B):
    return vanilla_option_price(S, K, T, rd, rf, sigma, 'call') - up_and_out_call(S, K, T, rd, rf, sigma, B)


def down_and_out_put(S, K, T, rd, rf, sigma, B):
    vanilla_price = vanilla_option_price(S, K, T, rd, rf, sigma, 'put')
    mu = (rd - rf - 0.5 * sigma**2) / sigma**2
    term2 = (B / S)**(2 * (mu + 1)) * vanilla_option_price(B**2 / S, K, T, rd, rf, sigma, 'put')
    return max(vanilla_price - term2,0)

def down_and_in_put(S, K, T, rd, rf, sigma, B):
    return vanilla_option_price(S, K, T, rd, rf, sigma, 'put') - down_and_out_put(S, K, T, rd, rf, sigma, B)

def up_and_out_put(S, K, T, rd, rf, sigma, B):
    vanilla_price = vanilla_option_price(S, K, T, rd, rf, sigma, 'put')
    mu = (rd - rf - 0.5 * sigma**2) / sigma**2
    term2 = (B / S)**(2 * (mu + 1)) * vanilla_option_price(B**2 / S, K, T, rd, rf, sigma, 'put')
    return max(vanilla_price - term2,0)

def up_and_in_put(S, K, T, rd, rf, sigma, B):
    return vanilla_option_price(S, K, T, rd, rf, sigma, 'put') - up_and_out_put(S, K, T, rd, rf, sigma, B)

In [12]:
def barrier_option_pricer(S, K, T, rd, rf, sigma, B, option_type='call', barrier_type='down-out'):
    if option_type == 'call':
        if barrier_type == 'down-out':
            return down_and_out_call(S, K, T, rd, rf, sigma, B)
        elif barrier_type == 'down-in':
            return down_and_in_call(S, K, T, rd, rf, sigma, B)
        elif barrier_type == 'up-out':
            return up_and_out_call(S, K, T, rd, rf, sigma, B)
        elif barrier_type == 'up-in':
            return up_and_in_call(S, K, T, rd, rf, sigma, B)
    elif option_type == 'put':
        if barrier_type == 'down-out':
            return down_and_out_put(S, K, T, rd, rf, sigma, B)
        elif barrier_type == 'down-in':
            return down_and_in_put(S, K, T, rd, rf, sigma, B)
        elif barrier_type == 'up-out':
            return up_and_out_put(S, K, T, rd, rf, sigma, B)
        elif barrier_type == 'up-in':
            return up_and_in_put(S, K, T, rd, rf, sigma, B)


In [13]:
# Example usage
S = 100  # Spot price
K = 100  # Strike price
T = 1    # Time to maturity (1 year)
rd = 0.05  # Domestic risk-free rate
rf = 0.02  # Foreign risk-free rate
sigma = 0.3  # Volatility
B = 95    # Barrier level

In [14]:


price = barrier_option_pricer(S, K, T, rd, rf, sigma, B, option_type='call', barrier_type='down-out')
print(f"The price of the down-and-out call option is: {price:.2f}")

price = barrier_option_pricer(S, K, T, rd, rf, sigma, B, option_type='call', barrier_type='down-in')
print(f"The price of the down-and-in call option is: {price:.2f}")

# vanilla option price
price = vanilla_option_price(S, K, T, rd, rf, sigma, 'call')
print(f"The price of the vanilla call option is: {price:.2f}")

The price of the down-and-out call option is: 5.74
The price of the down-and-in call option is: 7.28
The price of the vanilla call option is: 13.02


In [15]:
# for up and out call option  and up and in call option with vanilla option price
price = barrier_option_pricer(S, K, T, rd, rf, sigma, B, option_type='call', barrier_type='up-out')
print(f"The price of the up-and-out call option is: {price:.2f}")

price = barrier_option_pricer(S, K, T, rd, rf, sigma, B, option_type='call', barrier_type='up-in')
print(f"The price of the up-and-in call option is: {price:.2f}")

# vanilla option price
price = vanilla_option_price(S, K, T, rd, rf, sigma, 'call')
print(f"The price of the vanilla call option is: {price:.2f}")

The price of the up-and-out call option is: 2.59
The price of the up-and-in call option is: 10.43
The price of the vanilla call option is: 13.02


To get greeks of Vanilla options first and then move to greeks of corresponding barrier options using two methods

In [16]:
def vanilla_delta(S, K, T, rd, rf, sigma, option_type='call'):
    d1 = (np.log(S / K) + (rd - rf + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    if option_type == 'call':
        return np.exp(-rf * T) * norm.cdf(d1)
    elif option_type == 'put':
        return np.exp(-rf * T) * (norm.cdf(d1) - 1)

def vanilla_gamma(S, K, T, rd, rf, sigma):
    d1 = (np.log(S / K) + (rd - rf + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return np.exp(-rf * T) * norm.pdf(d1) / (S * sigma * np.sqrt(T))

def vanilla_vega(S, K, T, rd, rf, sigma):
    d1 = (np.log(S / K) + (rd - rf + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S * np.exp(-rf * T) * norm.pdf(d1) * np.sqrt(T)

def vanilla_theta(S, K, T, rd, rf, sigma, option_type='call'):
    d1 = (np.log(S / K) + (rd - rf + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        theta = - (S * sigma * np.exp(-rf * T) * norm.pdf(d1)) / (2 * np.sqrt(T)) - rd * K * np.exp(-rd * T) * norm.cdf(d2) + rf * S * np.exp(-rf * T) * norm.cdf(d1)
    elif option_type == 'put':
        theta = - (S * sigma * np.exp(-rf * T) * norm.pdf(d1)) / (2 * np.sqrt(T)) + rd * K * np.exp(-rd * T) * norm.cdf(-d2) - rf * S * np.exp(-rf * T) * norm.cdf(-d1)
    return theta

def vanilla_rho(S, K, T, rd, rf, sigma, option_type='call'):
    d1 = (np.log(S / K) + (rd - rf + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'call':
        rho = K * T * np.exp(-rd * T) * norm.cdf(d2)
    elif option_type == 'put':
        rho = -K * T * np.exp(-rd * T) * norm.cdf(-d2)
    return rho


In [17]:
def numerical_greek(greek, price_func, param_name, perturbation, *args, **kwargs):
    original_price = price_func(*args, **kwargs)
    perturb_args = list(args)
    param_index = ['S', 'K', 'T', 'rd', 'rf', 'sigma', 'B'].index(param_name)
    
    perturb_args[param_index] += perturbation
    perturbed_price_up = price_func(*perturb_args, **kwargs)
    
    perturb_args[param_index] -= 2 * perturbation
    perturbed_price_down = price_func(*perturb_args, **kwargs)
    
    if greek in ['delta', 'rho']:
        return (perturbed_price_up - original_price) / perturbation
    elif greek in ['gamma', 'vega']:
        return (perturbed_price_up - 2 * original_price + perturbed_price_down) / (perturbation ** 2)
    elif greek == 'theta':
        return (perturbed_price_down - original_price) / perturbation

# Example usage of numerical Greeks
perturbation = 0.01
price_func = barrier_option_pricer

delta = numerical_greek('delta', price_func, 'S', perturbation, S, K, T, rd, rf, sigma, B, option_type='call', barrier_type='down-out')
gamma = numerical_greek('gamma', price_func, 'S', perturbation, S, K, T, rd, rf, sigma, B, option_type='call', barrier_type='down-out')
vega = numerical_greek('vega', price_func, 'sigma', perturbation, S, K, T, rd, rf, sigma, B, option_type='call', barrier_type='down-out')
theta = numerical_greek('theta', price_func, 'T', perturbation, S, K, T, rd, rf, sigma, B, option_type='call', barrier_type='down-out')
rho = numerical_greek('rho', price_func, 'rd', perturbation, S, K, T, rd, rf, sigma, B, option_type='call', barrier_type='down-out')

print(f"Delta: {delta:.4f}")
print(f"Gamma: {gamma:.4f}")
print(f"Vega: {vega:.4f}")
print(f"Theta: {theta:.4f}")
print(f"Rho: {rho:.4f}")


Delta: 1.0844
Gamma: -0.0214
Vega: -3.9518
Theta: -1.1952
Rho: 23.9300


In [18]:

# Price of the down-and-out call option
price = barrier_option_pricer(S, K, T, rd, rf, sigma, B, option_type='call', barrier_type='down-out')
print(f"The price of the down-and-out call option is: {price:.2f}")

# Greeks calculation using numerical methods

# Delta
perturbation = 0.01
price_up = barrier_option_pricer(S + perturbation, K, T, rd, rf, sigma, B, option_type='call', barrier_type='down-out')
price_down = barrier_option_pricer(S - perturbation, K, T, rd, rf, sigma, B, option_type='call', barrier_type='down-out')
delta = (price_up - price_down) / (2*perturbation)
print(f"Delta: {delta:.4f}")

# Gamma
price_down = barrier_option_pricer(S - perturbation, K, T, rd, rf, sigma, B, option_type='call', barrier_type='down-out')
gamma = (price_up - 2 * price + price_down) / (perturbation ** 2)
print(f"Gamma: {gamma:.4f}")

# Vega
price_up = barrier_option_pricer(S, K, T, rd, rf, sigma + perturbation, B, option_type='call', barrier_type='down-out')
vega = (price_up - price) / perturbation
print(f"Vega: {vega:.4f}")

# Theta
perturbation_T = 1 / 365  # One day perturbation
price_down = barrier_option_pricer(S, K, T - perturbation_T, rd, rf, sigma, B, option_type='call', barrier_type='down-out')
theta = (price_down - price) / perturbation_T
print(f"Theta: {theta:.4f}")

# Rho
price_up = barrier_option_pricer(S, K, T, rd + perturbation, rf, sigma, B, option_type='call', barrier_type='down-out')
rho = (price_up - price) / perturbation
print(f"Rho: {rho:.4f}")

The price of the down-and-out call option is: 5.74
Delta: 1.0845
Gamma: -0.0214
Vega: 3.9591
Theta: -1.1917
Rho: 23.9300


In [19]:
import numpy as np
from scipy.stats import norm

def vanilla_option_price(S, K, T, rd, rf, sigma, option_type='call'):
    d1 = (np.log(S / K) + (rd - rf + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        price = S * np.exp(-rf * T) * norm.cdf(d1) - K * np.exp(-rd * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-rd * T) * norm.cdf(-d2) - S * np.exp(-rf * T) * norm.cdf(-d1)
    
    return price

def down_and_out_call(S, K, T, rd, rf, sigma, B):
    vanilla_price = vanilla_option_price(S, K, T, rd, rf, sigma, 'call')
    mu = (rd - rf - 0.5 * sigma**2) / sigma**2
    term2 = (B / S)**(2 * (mu + 1)) * vanilla_option_price(B**2 / S, K, T, rd, rf, sigma, 'call')
    return vanilla_price - term2

def up_and_out_call(S, K, T, rd, rf, sigma, B):
    vanilla_price = vanilla_option_price(S, K, T, rd, rf, sigma, 'call')
    mu = (rd - rf - 0.5 * sigma**2) / sigma**2
    term2 = (B / S)**(2 * mu) * vanilla_option_price(B, K, T, rd, rf, sigma, 'call')
    return vanilla_price - term2

def barrier_option_pricer(S, K, T, rd, rf, sigma, B, option_type='call', barrier_type='down-out'):
    if option_type == 'call':
        if barrier_type == 'down-out':
            return down_and_out_call(S, K, T, rd, rf, sigma, B)
        elif barrier_type == 'down-in':
            return vanilla_option_price(S, K, T, rd, rf, sigma, 'call') - down_and_out_call(S, K, T, rd, rf, sigma, B)
        elif barrier_type == 'up-out':
            return up_and_out_call(S, K, T, rd, rf, sigma, B)
        elif barrier_type == 'up-in':
            return vanilla_option_price(S, K, T, rd, rf, sigma, 'call') - up_and_out_call(S, K, T, rd, rf, sigma, B)
    elif option_type == 'put':
        # Implement other barrier option types if needed
        pass

# Example usage
S = 100  # Spot price
K = 100  # Strike price
T = 1    # Time to maturity (1 year)
rd = 0.05  # Domestic risk-free rate
rf = 0.02  # Foreign risk-free rate
sigma = 0.3  # Volatility
B = 95    # Barrier level

# Price of the down-and-out call option
price = barrier_option_pricer(S, K, T, rd, rf, sigma, B, option_type='call', barrier_type='down-out')
print(f"The price of the down-and-out call option is: {price:.2f}")

# Greeks calculation using numerical methods

# Delta
perturbation = 0.0001
price_up = barrier_option_pricer(S + perturbation, K, T, rd, rf, sigma, B, option_type='call', barrier_type='down-out')
delta = (price_up - price) / perturbation
print(f"Delta: {delta:.4f}")

# Gamma
price_down = barrier_option_pricer(S - perturbation, K, T, rd, rf, sigma, B, option_type='call', barrier_type='down-out')
gamma = (price_up - 2 * price + price_down) / (perturbation ** 2)
print(f"Gamma: {gamma:.4f}")

# Vega
price_up = barrier_option_pricer(S, K, T, rd, rf, sigma + perturbation, B, option_type='call', barrier_type='down-out')
vega = (price_up - price) / perturbation
print(f"Vega: {vega:.4f}")

# Theta
perturbation_T = 1 / 365  # One day perturbation
price_down = barrier_option_pricer(S, K, T - perturbation_T, rd, rf, sigma, B, option_type='call', barrier_type='down-out')
theta = (price_down - price) / perturbation_T
print(f"Theta: {theta:.4f}")

# Rho
price_up = barrier_option_pricer(S, K, T, rd + perturbation, rf, sigma, B, option_type='call', barrier_type='down-out')
rho = (price_up - price) / perturbation
print(f"Rho: {rho:.4f}")


The price of the down-and-out call option is: 5.74
Delta: 1.0845
Gamma: -0.0214
Vega: 3.9778
Theta: -1.1917
Rho: 23.6169


In [20]:
import numpy as np
from scipy.stats import norm

def vanilla_option_price(S, K, T, rd, rf, sigma, option_type='call'):
    d1 = (np.log(S / K) + (rd - rf + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == 'call':
        price = S * np.exp(-rf * T) * norm.cdf(d1) - K * np.exp(-rd * T) * norm.cdf(d2)
    elif option_type == 'put':
        price = K * np.exp(-rd * T) * norm.cdf(-d2) - S * np.exp(-rf * T) * norm.cdf(-d1)
    
    return price

def vanilla_delta(S, K, T, rd, rf, sigma, option_type='call'):
    d1 = (np.log(S / K) + (rd - rf + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    if option_type == 'call':
        delta = np.exp(-rf * T) * norm.cdf(d1)
    elif option_type == 'put':
        delta = np.exp(-rf * T) * (norm.cdf(d1) - 1)
    return delta

def down_and_out_call(S, K, T, rd, rf, sigma, B):
    vanilla_price = vanilla_option_price(S, K, T, rd, rf, sigma, 'call')
    mu = (rd - rf - 0.5 * sigma**2) / sigma**2
    adjusted_price = vanilla_option_price(B**2 / S, K, T, rd, rf, sigma, 'call')
    term2 = (B / S)**(2 * (mu + 1)) * adjusted_price
    return vanilla_price - term2

def down_and_out_call_delta(S, K, T, rd, rf, sigma, B):
    # Vanilla call delta
    delta_vanilla = vanilla_delta(S, K, T, rd, rf, sigma, 'call')
    
    # Adjustment term delta
    mu = (rd - rf - 0.5 * sigma**2) / sigma**2
    adjusted_price_delta = vanilla_delta(B**2 / S, K, T, rd, rf, sigma, 'call')
    term2_delta = (B / S)**(2 * (mu + 1)) * (-2 * (mu + 1) / S) * adjusted_price_delta
    
    # Total delta
    delta_down_and_out = delta_vanilla + term2_delta
    
    return delta_vanilla, term2_delta, delta_down_and_out

# Example usage
S = 100  # Spot price
K = 100  # Strike price
T = 1    # Time to maturity (1 year)
rd = 0.05  # Domestic risk-free rate
rf = 0.02  # Foreign risk-free rate
sigma = 0.3  # Volatility
B = 95    # Barrier level

# Price of the down-and-out call option
price = down_and_out_call(S, K, T, rd, rf, sigma, B)
print(f"The price of the down-and-out call option is: {price:.2f}")

# Delta calculation for the down-and-out call option
delta_vanilla, term2_delta, delta_down_and_out = down_and_out_call_delta(S, K, T, rd, rf, sigma, B)
print(f"Delta (Vanilla): {delta_vanilla:.4f}")
print(f"Delta (Adjustment Term): {term2_delta:.4f}")
print(f"Delta (Total): {delta_down_and_out:.4f}")


The price of the down-and-out call option is: 5.74
Delta (Vanilla): 0.5869
Delta (Adjustment Term): -0.0069
Delta (Total): 0.5799
